https://www.cs.toronto.edu/~tingwuwang/REINFORCE.pdf

- 基于神经网络（将策略模型化）的策略表示为 $\pi_\theta(a|s)$
- 考虑回合制任务，并基于策略 $\pi_\theta$ 选择行动
    - $\tau=(S_0,A_0,R_0,S_1,A_1,R_1,\cdots),S_{T+1}$
    - 使用折现率 $\gamma$ 对收益 return
        - $G(\tau)=R_0+\gamma R_1+\gamma^2R_2+\cdots+\gamma^TR_T$
- 此时目标函数 $J(\theta)$（最大化）表示为：$J(\theta)=\mathbb E_{\tau\sim \pi_\theta}[G(\tau)]$
- $\nabla_\theta J(\theta)=\nabla_\theta\mathbb E_{\tau\sim\pi_\theta}[G(\tau)]=\mathbb E_{\tau\sim\pi_\theta}[\sum_{t=0}^TG(\tau)\nabla_\theta\log\pi_\theta(A_t|S_t)]$